In [ ]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [ ]:
import openai as client

assistant_id = "asst_n0oq9SJxSpjOJvh6bouZfmDp"

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-1106-preview",
#     tools=functions,
# )

In [5]:
import openai


# Thread 먼저 생성
thread = openai.beta.threads.create()

# 메시지를 따로 추가
openai.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I want to know if the Salesforce stock is a good buy",
)

C:\Users\dtq10\AppData\Local\Temp\ipykernel_24308\3839882613.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = openai.beta.threads.create()
C:\Users\dtq10\AppData\Local\Temp\ipykernel_24308\3839882613.py:8: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  openai.beta.threads.messages.create(


Message(id='msg_psUY2JIBR86qBZfYEj3BGqRp', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I want to know if the Salesforce stock is a good buy'), type='text')], created_at=1749916330, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_47J7hC8eImg7oaIqyUod8t9Q')

In [6]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

C:\Users\dtq10\AppData\Local\Temp\ipykernel_24308\1531244780.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


Run(id='run_OOvvUO143VYKoNer5qTCFbUw', assistant_id='asst_n0oq9SJxSpjOJvh6bouZfmDp', cancelled_at=None, completed_at=None, created_at=1749916433, expires_at=1749917033, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_47J7hC8eImg7oaIqyUod8t9Q', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=Functio

In [7]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outpus,
    )

In [8]:
get_tool_outputs(run.id, thread.id)

C:\Users\dtq10\AppData\Local\Temp\ipykernel_24308\3435834180.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.runs.retrieve(


Calling function: get_ticker with arg {"company_name":"Salesforce"}


[{'output': "Get a real-time Salesforce, Inc. (CRM) stock price with an overview of financials, statistics, forecasts, charts and more. ... Ticker Symbol CRM. Website https://www.salesforce.com. Full Company Profile. Financial Performance. In 2024, Salesforce's revenue was $37.90 billion, an increase of 8.72% compared to the previous year's $34.86 billion ... Historical CRM Stock Quote. CRM Stock Chart. follow us on facebook; follow us on twitter; follow us on linkedin; follow us on instagram; follow us on youtube; follow us on tiktok; ... Various trademarks held by their respective owners. Salesforce, Inc. Salesforce Tower, 415 Mission Street, 3rd Floor, San Francisco, CA 94105, United States ... See the latest Salesforce stock price NYSE: CRM stock ratings, related news, valuation, dividends and more to help you make your investing decisions. Shares Outstanding (Ticker) 956.00M. CRM Ownership. Risk. Short Interest. 1.40%. Shares Short. 13.38M. Days to Cover. ... Salesforce, Inc.'s st

In [19]:
submit_tool_outputs(run.id, thread.id)

C:\Users\dtq10\AppData\Local\Temp\ipykernel_24308\3435834180.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.runs.retrieve(


Calling function: get_income_statement with arg {"ticker": "CRM"}
Calling function: get_balance_sheet with arg {"ticker": "CRM"}
Calling function: get_daily_stock_performance with arg {"ticker": "CRM"}


C:\Users\dtq10\AppData\Local\Temp\ipykernel_24308\3435834180.py:40: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.runs.submit_tool_outputs(


Run(id='run_OOvvUO143VYKoNer5qTCFbUw', assistant_id='asst_n0oq9SJxSpjOJvh6bouZfmDp', cancelled_at=None, completed_at=None, created_at=1749916433, expires_at=1749917033, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1749916487, status='queued', thread_id='thread_47J7hC8eImg7oaIqyUod8t9Q', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=F

In [21]:
get_run(run.id, thread.id).status

C:\Users\dtq10\AppData\Local\Temp\ipykernel_24308\3435834180.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  return client.beta.threads.runs.retrieve(


'completed'

In [22]:
get_messages(thread.id)

C:\Users\dtq10\AppData\Local\Temp\ipykernel_24308\3435834180.py:15: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = client.beta.threads.messages.list(thread_id=thread_id)


user: I want to know if the Salesforce stock is a good buy
assistant: To analyze whether Salesforce (CRM) stock is a good buy, we need to consider its financial data and stock performance. Based on the income statement and balance sheet provided for the last four fiscal year periods, and the daily stock performance over the last 100 days, here's a summary of the key indicators:

### Income Statement Highlights
- Revenue has been increasing, with the latest total revenue at $37.89 billion.
- Net income also showed growth to $6.197 billion in the most recent year.
- Research and development, as well as selling, general and administrative expenses, have been significant but consistent with the growth trend.

### Balance Sheet Highlights
- There is a healthy amount of current assets ($29.73 billion) to cover current liabilities ($27.98 billion), indicating a stable working capital situation.
- It has a substantial amount of goodwill on its balance sheet ($51.28 billion), suggesting previou